## Multi-Function Call

**This tutorial is available in English and is attached below the Chinese explanation**

本章节，我将使用 `GLM-4-Plus` 模型演示如何同时调用多个函数。多工具调用在效率上有很大的提升，可以减少多轮对话的次数，提高对话的效率。`GLM-4-Plus` 模型支持同时调用多个函数，包括同时调用不同的函数、同时调用同一个函数多次等。

请注意，只有 `GLM-4-Plus` 模型支持该功能。在开始之前，建议您先阅读 [Function Call](glm_function_call.ipynb) 教程。

In this CookBook, I will demonstrate how to call multiple functions at the same time using the `GLM-4-Plus` model. Multi-tool calls can greatly improve efficiency, reduce the number of rounds of conversation, and improve the efficiency of the conversation. The `GLM-4-Plus` model supports calling multiple functions at the same time, including calling different functions at the same time, calling the same function multiple times at the same time, etc.

Please note that only the `GLM-4-Plus` model supports this feature. Before you begin, it is recommended that you read the [Function Call](glm_function_call.ipynb) tutorial.

## 1. 配置API Key

请确保按照 `requirements.txt`中安装了正确的依赖，接着我们需要配置 API Key，这个 API Key 可以在 [ZhipuAI API 平台申请](https://open.bigmodel.cn/) 。我们可以通过以下方式配置 API Key

Please make sure that the correct dependencies are installed according to `requirements.txt`. Then we need to configure the API Key, which can be applied for on the [ZhipuAI API platform](https://open.bigmodel.cn/). We can configure the API Key in the following ways

In [1]:
from zhipuai import ZhipuAI
import json

client = ZhipuAI(api_key="your api key")

# 2. 同时调用不同的两个函数。

GLM-4-Plus 支持同时调用不同的两个函数。下面的例子中，我们将调用 `get_weather` 函数和 `get_coordinates` 函数，分别查询北京的天气情况和坐标。当用户询问天气和坐标时，我们可以同时调用这两个函数，然后将结果一起返回给用户。

In [2]:
def get_weather(location, unit):
    if location == "Beijing":
        if unit == "c":
            return {"location": location, "temperature": "20°C"}
        elif unit == "f":
            return {"location": location, "temperature": "68°F"}
    else:
        return {"error": "Location not found"}


def get_coordinates(address, region):
    if address == "Beijing":
        if region == "world":
            return {"address": address, "coordinates": {"lat": 39.9042, "lon": 116.4074}}
        elif region == "local":
            return {"address": address, "coordinates": {"lat": 39.9, "lon": 116.4}}
    else:
        return {"error": "Address not found"}


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                    "unit": {"type": "string", "enum": ["c", "f"]},
                },
                "required": ["location", "unit"],
                "additionalProperties": False,
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_coordinates",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "address": {"type": "string"},
                    "region": {"type": "string", "enum": ["world", "local"]},
                },
                "required": ["address", "region"],
                "additionalProperties": False
            }
        }
    }
]

messages = [
    {"role": "user", "content": "What's the weather like in Beijing today and where is beijing?"}]
response = client.chat.completions.create(
    model="glm-4-plus",
    messages=messages,
    tools=tools,
    tool_choice="required"
)

print(response)

Completion(model='glm-4-plus', created=1727793872, choices=[CompletionChoice(index=0, finish_reason='tool_calls', message=CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_9073576668991404245', function=Function(arguments='{"location": "Beijing", "unit": "c"}', name='get_weather'), type='function', index=0), CompletionMessageToolCall(id='call_9073576668991404246', function=Function(arguments='{"address": "Beijing", "region": "world"}', name='get_coordinates'), type='function', index=1)]))], request_id='20241001224431d422ef43fcdf4bcd', id='20241001224431d422ef43fcdf4bcd', usage=CompletionUsage(prompt_tokens=301, completion_tokens=34, total_tokens=335))


In [3]:
response = response.choices[0].dict()

我们将直接执行模型的返回，并调用工具，在工具中，我们写死了北京的数据。接着，我们将两个模型回答一起提交给 API。请注意，ID号码和顺序必须与模型工具调用的顺序完全一致，这样才能保证模型正常返回。在这里，我写了一个简单的执行模型的函数，根据函数名调用对应的函数。

We will execute the model's return directly and call the tool. In the tool, we hard-coded the data for Beijing. Then, we will submit the two model answers together to the API. Please note that the ID numbers and order must be exactly the same as the order of the model tool calls to ensure that the model returns normally. Here, I wrote a simple function to execute the model and call the corresponding function according to the function name.

In [4]:
tool_messages = [
    response['message'],
    {
        "role": "tool",
        "content": None,
        "tool_call_id": response['message']['tool_calls'][0]['id']
    },
    {
        "role": "tool",
        "content": None,
        "tool_call_id": response['message']['tool_calls'][1]['id']
    }
]


def run_function(function, parameters):
    if function == "get_weather":
        return get_weather(**parameters)
    elif function == "get_coordinates":
        return get_coordinates(**parameters)
    else:
        return {"error": "Function not found"}


for i, tool_call in enumerate(response['message']['tool_calls']):
    arguments = tool_call['function']['arguments']
    if isinstance(arguments, str):
        arguments = json.loads(arguments)
    tool_messages[i + 1]['content'] = json.dumps(run_function(tool_call['function']['name'], arguments))



将消息进行拼接，得到完整信息。

Concatenate the messages to get the complete information.

In [5]:
messages +=  tool_messages
messages

[{'role': 'user',
  'content': "What's the weather like in Beijing today and where is beijing?"},
 {'content': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_9073576668991404245',
    'function': {'arguments': '{"location": "Beijing", "unit": "c"}',
     'name': 'get_weather'},
    'type': 'function',
    'index': 0},
   {'id': 'call_9073576668991404246',
    'function': {'arguments': '{"address": "Beijing", "region": "world"}',
     'name': 'get_coordinates'},
    'type': 'function',
    'index': 1}]},
 {'role': 'tool',
  'content': '{"location": "Beijing", "temperature": "20\\u00b0C"}',
  'tool_call_id': 'call_9073576668991404245'},
 {'role': 'tool',
  'content': '{"address": "Beijing", "coordinates": {"lat": 39.9042, "lon": 116.4074}}',
  'tool_call_id': 'call_9073576668991404246'}]

调用模型返回，我们可以看到模型返回了北京的天气和坐标。

Call the model to return, we can see that the model returns the weather and coordinates of Beijing.

In [6]:
response = client.chat.completions.create(
    model="glm-4-plus",
    messages=messages,
)

print(response)

Completion(model='glm-4-plus', created=1727793874, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='The weather in Beijing today is 20°C. Beijing is located at coordinates 39.9042 latitude and 116.4074 longitude.', role='assistant', tool_calls=None))], request_id='2024100122443242d30402158c4325', id='2024100122443242d30402158c4325', usage=CompletionUsage(prompt_tokens=105, completion_tokens=33, total_tokens=138))


# 3. 同一个函数同时调用多次

GLM-4-Plus 也支持使用同时并行调用同一个函数多次，使用不同函数。下面的例子中，我们将调用 `check_weather` 函数四次，分别查询纽约、伦敦、北京、东京的天气情况。

In [7]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "check_weather",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                    "unit": {"type": "string", "enum": ["c", "f"]},
                },
                "required": ["location"],
                "additionalProperties": False,
            },
        },
    },
]
messages = [
    {"role": "system", "content": "You are a helpful assistant providing weather updates."},
    {"role": "user", "content": "What's the weather like in New York, London, Tokyo and Beijing?"}]

response = client.chat.completions.create(
    model="glm-4-plus",
    messages=messages,
    tools=tools,
    tool_choice="required"
)

print(response)

Completion(model='glm-4-plus', created=1727793877, choices=[CompletionChoice(index=0, finish_reason='tool_calls', message=CompletionMessage(content=None, role='assistant', tool_calls=[CompletionMessageToolCall(id='call_9073574985363290468', function=Function(arguments='{"location": "New York", "unit": "c"}', name='check_weather'), type='function', index=0), CompletionMessageToolCall(id='call_9073574985363290469', function=Function(arguments='{"location": "London", "unit": "c"}', name='check_weather'), type='function', index=1), CompletionMessageToolCall(id='call_9073574985363290470', function=Function(arguments='{"location": "Tokyo", "unit": "c"}', name='check_weather'), type='function', index=2), CompletionMessageToolCall(id='call_9073574985363290471', function=Function(arguments='{"location": "Beijing", "unit": "c"}', name='check_weather'), type='function', index=3)]))], request_id='202410012244343be7c76212a64095', id='202410012244343be7c76212a64095', usage=CompletionUsage(prompt_tok

In [8]:
response = response.choices[0].dict()

我们将提前准备好的回答，分别填入对应的 `tool_call_id` 中，然后将这些回答一起提交给 API。请注意，ID号码和顺序必须与模型工具调用的顺序完全一致，这样才能保证模型正常返回。

In [9]:
weather_data = {
    "New York": {"temperature": "22°C", "condition": "Sunny"},
    "London": {"temperature": "15°C", "condition": "Cloudy"},
    "Tokyo": {"temperature": "25°C", "condition": "Rainy"},
    "Beijing": {"temperature": "-1°C", "condition": "Cloudy"}
}

messages += [
    response['message'],
    {
        "role": "tool",
        "content": json.dumps({
            "city": "New York",
            "weather": weather_data["New York"]
        }),
        # Here we specify the tool_call_id that this result corresponds to
        "tool_call_id": response['message']['tool_calls'][0]['id']
    },
    {
        "role": "tool",
        "content": json.dumps({
            "city": "London",
            "weather": weather_data["London"]
        }),
        "tool_call_id": response['message']['tool_calls'][1]['id']
    },
    {
        "role": "tool",
        "content": json.dumps({
            "city": "Tokyo",
            "weather": weather_data["Tokyo"]
        }),
        "tool_call_id": response['message']['tool_calls'][2]['id']
    },
    {
        "role": "tool",
        "content": json.dumps({
            "city": "Beijing",
            "weather": weather_data["Beijing"]
        }),
        "tool_call_id": response['message']['tool_calls'][3]['id']
    }
]

打印消息，我们看到每个工具调用的结果已经返回，并拼接到了消息中。

print the messages, we can see that the results of each tool call have been returned and appended to the messages.

In [10]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant providing weather updates.'},
 {'role': 'user',
  'content': "What's the weather like in New York, London, Tokyo and Beijing?"},
 {'content': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_9073574985363290468',
    'function': {'arguments': '{"location": "New York", "unit": "c"}',
     'name': 'check_weather'},
    'type': 'function',
    'index': 0},
   {'id': 'call_9073574985363290469',
    'function': {'arguments': '{"location": "London", "unit": "c"}',
     'name': 'check_weather'},
    'type': 'function',
    'index': 1},
   {'id': 'call_9073574985363290470',
    'function': {'arguments': '{"location": "Tokyo", "unit": "c"}',
     'name': 'check_weather'},
    'type': 'function',
    'index': 2},
   {'id': 'call_9073574985363290471',
    'function': {'arguments': '{"location": "Beijing", "unit": "c"}',
     'name': 'check_weather'},
    'type': 'function',
    'index': 3}]},
 {'role': 'tool',
  'content': '{

调用模型返回，我们可以看到模型返回了四个城市的天气情况。

Call the model to return, we can see that the model returns the weather conditions of four cities.

In [11]:
response = client.chat.completions.create(
    model="glm-4-plus",
    messages=messages,
)

print(response)

Completion(model='glm-4-plus', created=1727793882, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content="Here's the latest weather update for the cities you asked about:\n\n- **New York**: It's sunny with a temperature of 22°C.\n- **London**: The weather is cloudy and the temperature is 15°C.\n- **Tokyo**: It's rainy there, with a temperature of 25°C.\n- **Beijing**: The sky is cloudy and the temperature is -1°C.\n\nStay warm or cool as needed! 🌤️🌧️❄️☀️", role='assistant', tool_calls=None))], request_id='202410012244375aa8f5caa0cc477e', id='202410012244375aa8f5caa0cc477e', usage=CompletionUsage(prompt_tokens=217, completion_tokens=103, total_tokens=320))


# 4. Conclusion

大模型是如何完成多个工具同时调用的：

**步骤 1**：向模型提供可能导致模型选择使用工具的提示。工具的描述（如函数名称和签名）在“工具”列表中定义，并在 API 调用中传递给模型。如果模型选择了某个工具，函数名称和参数将包含在响应中。<br>

**步骤 2**：程序化地检查模型是否想要调用某个函数。如果为真，继续执行步骤 3。<br>  
**步骤 3**：从响应中提取函数名称和参数，使用这些参数调用函数。将结果附加到消息列表中。<br>    
**步骤 4**：使用消息列表再次调用 chat completions API 以获取响应。

通过以上路线，大模型能更好的处理多个工具调用的情况，提高对话效率，随着工具数量的增加，对话效率会更加明显的提升。

The LLM is how to complete multiple tool calls at the same time:

**Step 1**: Prompt the model with content that may result in model selecting a tool to use. The description of the tools such as a function names and signature is defined in the 'Tools' list and passed to the model in API call. If selected, the function name and parameters are included in the response.<br>
  
**Step 2**: Check programmatically if model wanted to call a function. If true, proceed to step 3. <br>  
**Step 3**: Extract the function name and parameters from response, call the function with parameters. Append the result to messages. <br>    
**Step 4**: Invoke the chat completions API with the message list to get the response. 

Through the above route, the LLM can better handle the situation of multiple tool calls, improve the efficiency of the conversation, and with the increase of the number of tools, the efficiency of the conversation will be more obvious improvement.